In [2]:
# The model that you want to train from the Hugging Face hub
model_name = "mistralai/Mistral-7B-Instruct-v0.1"



# Fine-tuned model name
new_model = "StarkWizard/Mistral-7b-instruct-cairo-PEFT"



import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

hub_name = "StarkWizard/Mistral-7b-instruct-cairo-instruct"

max_steps = 1000 # to tweak to get the best out of the model 

Push Model To Hub

- Make sure to reinit the kernel
- We reload the base model
- Load the Peft adapter
- Push the merged model

NOTE:

We use CPU because of the error: Cannot merge LORA layers when the model is loaded in 8-bit mode
Merging is fast, so it's not an issue

In [3]:

from transformers import AutoModelForCausalLM
from peft import PeftModel
import torch
from transformers import  AutoTokenizer, BitsAndBytesConfig

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name,
                                             trust_remote_code=True,
                                             low_cpu_mem_usage=True,
                                             device_map={"": "cpu"},
                                             
                                             )

model_to_merge  = PeftModel.from_pretrained(model, new_model,
                        torch_dtype=torch.float16, 
                        device_map={"": "cpu"}
                         )
merged_model = model_to_merge.merge_and_unload()
#model.save_pretrained("cairo-mistral")
merged_model.push_to_hub(hub_name,max_shard_size="1GB")
tokenizer.push_to_hub(hub_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/StarkWizard/Mistral-7b-instruct-cairo-instruct/commit/259eddc616188dbaecc25ea6a2c5fe9e6b7ee599', commit_message='Upload tokenizer', commit_description='', oid='259eddc616188dbaecc25ea6a2c5fe9e6b7ee599', pr_url=None, pr_revision=None, pr_num=None)

---

Quick inference test to check the model

Make sure you unload reload the kernel before restarting

Load model from hub for inference

- If you just need inference, run this
- we load the model from HFace Hub in 4 bits


In [4]:
import torch
from transformers import AutoTokenizer, TextStreamer, GenerationConfig, BitsAndBytesConfig
from attention_sinks import AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=hub_name,
                                             trust_remote_code=True,
                                             device_map={"": 0},
                                             attention_sink_size=4,
                                             quantization_config=bnb_config,
                                            attention_sink_window_size=252, # <- Low for the sake of faster generation
                                             )
model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

[Attention Sinks] Injected Position Shifting into 32 attention classes.
[Attention Sinks] Injected Attention Sink KV Cache into 1 model class.


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )

Votre GPU supporte bfloat16!


In [11]:
from transformers import TextStreamer, GenerationConfig


#prompt = "Create an array and append 3 domestic animal names as string"
#prompt = "give an exemple of constructor"
#prompt = "create a felt and affect it a v/alue of 1"
#prompt="In the example 'let y = { let x = 3; x + 1 };', what is the value of y? give a similar sample"
#prompt="How can you print a variable's value in Cairo? Give a full sample using an array"
#prompt = "hat type of operations can be considered expressions in Cairo? Give an exemple."
#prompt = "What is a characteristic feature of Felt252Dict<T> when interacting with it? Give a sample"
#prompt = "what are spans used for"
#prompt = "How do I know if an array is empty, give 2 examples"
#prompt = "what makes Cairo special"
#prompt = "Create an array and append some domestic animal names"
#prompt="write a contract that computes the fibonacci of caller's address and explain the weakness of the program"
#prompt="write a contract that returns the fibonacci of the caller address. Provide the steps to compile and deploy"
#prompt="write an empty contract template and give the instructions to compile and deploy it"
#prompt="write a contract that stores the caller's address in its constructor and a function that returns the fibonnacci of that address"
#prompt="How can you implement the `Copy` trait on a custom type in Cairo?"
#prompt = "What's the significance of the `self` parameter in my method of the `CubeGeom` trait?"
#prompt = "write a contract with a function that returns the address of the caller"
prompt="how can i cast a variable into another ?"

prompt = "What do the terms 'parent' and 'child' mean in the context of modules?"
prompt="What can be defined in trait and impl blocks related to a type?"
prompt = "How do I know if an array is empty, give 2 examples"

prompt="write a  basic simple contract that returns the fibonacci of a number"
prompt = "Create an array and append 3 domestic animal names as string"



#prompt = "How do I know if an array is empty, give 2 examples "

#prompt = "Provide a basic code snippet that defines a struct named 'User' with fields like 'active', 'username', and so on"


#prompt="write an empty contract template and give the instructions to compile and deploy it"

#prompt = "write a contract with a function that returns the address of the caller"
#prompt = "write a contract with a add_felt function that gets the sum of 2 felt parameters"
#prompt = "Create an array and append 3 domestic animal names as string"
prompt = "write a contract that returns the fibonacci of the caller address"

prompt="Can you provide a code snippet that declares an external function which can modify the contract's state?"
prompt = "What's the difference between Nullable<T> and Option in terms of where the wrapped value is stored?"

prompt = "How does the Cairo convention style name functions and variables?"

prompt = "write a contract that returns the fibonacci of the caller address"

prompt = "What does the #[starknet::component] attribute signify?"

prompt = " What is the naming convention for enum variants, give some examples"

#prompt = "write a contract for implementing a stack with push pop len functions"
prompt = "write a contract that returns the fibonacci of the caller's address"

text =f"""[INST]
<<SYS>>
A student asks you a question about Cairo 1. Provide a concise answer to the student's questions,do not expand the subject of the question, do not introduce any new topics or new question not provided by the student.
Make sure the explanations never be longer than 300 words.Don’t justify your answers. Don’t give information not mentioned in the CONTEXT INFORMATION.provide only one solution <SYS>>

Question: I'm working in Cairo 1 :{prompt} 
[/INST]"""

input_ids = tokenizer.encode(text, return_tensors="pt").to(model.device)

with torch.no_grad():
    streamer = TextStreamer(tokenizer)
    generated_tokens = model.generate(
        input_ids,
        generation_config=GenerationConfig(
            # use_cache=True is required, the rest can be changed up.
            use_cache=True,
            min_new_tokens=1,
            max_new_tokens=1050,

                                early_stopping=True,
                                bos_token_id=model.config.bos_token_id,
                                eos_token_id=model.config.eos_token_id,
                                pad_token_id=model.config.eos_token_id,
    temperature=0.0007,
    top_p=0.95,
    repetition_penalty=1.15,

                               do_sample=True
        ),
        streamer=streamer,
    )
    # Decode the final generated text
    output_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

<s> [INST]
<<SYS>>
A student asks you a question about Cairo 1. Provide a concise answer to the student's questions,do not expand the subject of the question, do not introduce any new topics or new question not provided by the student.
Make sure the explanations never be longer than 300 words.Don’t justify your answers. Don’t give information not mentioned in the CONTEXT INFORMATION.provide only one solution <SYS>>

Question: I'm working in Cairo 1 :write a contract that returns the fibonacci of the caller's address 
[/INST] To write a contract that returns the Fibonacci number of the caller's address, you can use the following code:
```cairo
#[starknet::contract]
mod fib_address {
   #[external(v0)]
   fn get_fibonacci() -> u64 {
       let caller = get_caller();
       return fibonacci(caller);
   }
   
   #[internal]
   fn fibonacci(address: felt252) -> u64 {
       if address == 0 || address == 1 {
           1
       } else {
           fibonacci(address - 1) + fibonacci(address -

In [10]:
from transformers import TextStreamer, GenerationConfig


prompt = "Create an array and append domestic animal names"
prompt = "give an exemple of a simplestorage contract with unit tests"
#prompt = "create a felt and affect it a value of 1"
#prompt="In the example 'let y = { let x = 3; x + 1 };', what is the value of y? give a similar sample"
#prompt="How can you print a variable's value in Cairo? Give a full sample using an array"
#prompt = "hat type of operations can be considered expressions in Cairo? Give an exemple."
#prompt = "What is a characteristic feature of Felt252Dict<T> when interacting with it? Give a sample"
#prompt = "How do I create a span ? "
#prompt = "How do I know if an array is empty, give 2 examples"
#prompt = "what makes Cairo special"
#prompt = "Create an array and append some domestic animal names"
#prompt="write a contract that computes the fibonacci of caller's address and explain the weakness of the program"
#prompt="write a contract that returns the fibonacci of a value passed as parameter. implement the test function for the value 10"
#prompt="write an empty contract template and give the instructions to compile and deploy it"
#prompt="write a contract that stores the caller's address in its constructor and a function that returns the fibonnacci of that address"
#prompt="How can you implement the `Copy` trait on a custom type in Cairo?"
#prompt = "What's the significance of the `self` parameter in my method of the `CubeGeom` trait?"
#prompt = "write a contract with a function that returns the address of the caller"
#prompt="how can i cast a variable into another ?"

#prompt = "What do the terms 'parent' and 'child' mean in the context of modules?"
#prompt="What can be defined in trait and impl blocks related to a type?"
#prompt = "How do I know if an array is empty, give 2 examples"

#rompt="write a  basic simple contract that returns the fibonacci of the caller's address"
#prompt = "Create an array and append some domestic animal names as string"



#prompt = "How do I know if an array is empty, give 2 examples "

#prompt = "Provide a basic code snippet that defines a struct named 'User' with fields like 'active', 'username', and so on"


#prompt="write an empty contract template and give the instructions to compile and deploy it"

#prompt = "write a contract with a function that returns the address of the caller"
#prompt = "write a contract with a add_felt function that gets the sum of 2 felt parameters"
#prompt = "Create an array and append 3 domestic animal names as string and print the the second element"
#prompt = "write a contract that returns the fibonacci of the caller address"

prompt="write a  contract that provides a function that takes two arrays as parameter and store an array composed of the sum of the element of the arrays"
#prompt = "What's the difference between Nullable<T> and Option in terms of where the wrapped value is stored?"


#prompt = "How does the Cairo convention style name functions and variables?"

#prompt = "write a  contract that returns the fibonacci(caller address)"

#prompt = "What does the #[starknet::component] attribute signify?"

#prompt = " What is the naming convention for enum variants, give some examples"

#prompt ="""#[test]
#[available_gas(200000)]
#fn test_loop() {
#    let mut counter = 0;
#    //TODO make the test pass without changing any existing line
#    loop {
#        counter += 1;
#    };
#    assert(counter == 140, 'counter should be 140')
#}
#"""

prompt = "write a contract for implementing a function that computes u(n)=u(n-1)+1, with n passed as parameter"

text =f"""[INST]
<<SYS>>
A student asks you a question about Cairo 1. Provide a concise answer to the student's questions,do not expand the subject of the question, do not introduce any new topics or new question not provided by the student.
Make sure the explanations never be longer than 300 words.Don’t justify your answers. Don’t give information not mentioned in the CONTEXT INFORMATION.provide only one solution <SYS>>

Question: I'm working in Cairo 1 :{prompt} 
[/INST]"""

input_ids = tokenizer.encode(text, return_tensors="pt").to(model.device)

with torch.no_grad():
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    generated_tokens = model.generate(
        input_ids,
        generation_config=GenerationConfig(
                       # use_cache=True is required, the rest can be changed up.
            use_cache=True,
            min_new_tokens=1,
            max_new_tokens=1050,
            penalty_alpha=0.1,
            top_k=50,
            do_sample=True,
            top_p=0.99,

            repetition_penalty =1.2,
            temperature=0.1,
                        early_stopping=True,
            bos_token_id=model.config.bos_token_id,
            eos_token_id=model.config.eos_token_id,
            pad_token_id=model.config.eos_token_id,
        ),
        streamer=streamer,
    )
    # Decode the final generated text
    output_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

```cairo
#[starknet::contract]
mod U {
   #[external(v0)]
   fn compute_u(self: @ContractState, n: felt252) -> (felt252, felt252) {
       let mut u = self.state.read();
       u.append(n);
       (u, u)
   }
}
```


In [136]:
prompt = "give an exemple of a simplestorage contract with unit tests"

text =f"""[INST]
<<SYS>>
A student asks you a question about Cairo 1. Provide a concise answer to the student's questions,do not expand the subject of the question, do not introduce any new topics or new question not provided by the student.
Make sure the explanations never be longer than 300 words.Don’t justify your answers. Don’t give information not mentioned in the CONTEXT INFORMATION.provide only one solution <SYS>>

Question: I'm working in Cairo 1 :{prompt} 
[/INST]"""

input_ids = tokenizer.encode(text, return_tensors="pt").to(model.device)

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
t = model.generate(input_ids, streamer=streamer, max_new_tokens=800)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



```
#[starknet::contract]
mod Storage {
   #[storage]
   struct Storage {
       stored_data: u128
   }

   #[external(v0)]
   fn set(ref self, data: u128) {
       self.stored_data.write(data);
   }

   #[external(v0)]
   fn get(ref self) -> u128 {
       self.stored_data.read()
   }
}

#[cfg(test)]
mod tests {
   use super::Storage;

   #[test]
   fn test_set_and_get() {
       let mut storage = Storage::default();
       storage.set(100);
       assert(storage.get() == 100, 'stored data is not correct');
   }
}
```




























































































KeyboardInterrupt: 